In [1]:
%run __init__.py

In [10]:
from pydantic import BaseModel, Field, conint, model_validator, ValidationError
from ipyautoui import AutoUi

class ApertureSettings(BaseModel):
    radius : conint(ge=1) = Field(default=1)
    inner_annulus : conint(ge=1) = Field(default=2)
    outer_annulus : conint(ge=1) = Field(default=3)

    class Config:
        validate_assignment = True
        validate_default = True

    @model_validator(mode="after")
    def check_annuli(cls, values):
        if values.inner_annulus >= values.outer_annulus:
            raise ValueError('inner_annulus must be smaller than outer_annulus')
        if values.radius >= values.inner_annulus:
            raise ValueError('radius must be smaller than inner_annulus')
        return values

In [11]:
ui = AutoUi(ApertureSettings)

counter = 0
def my_observer(change):
    global counter
    counter += 1
    print(f"My observer call {counter}")
    print(ui.value)
    try: 
        ApertureSettings(**ui.value)
    except ValidationError:
        print("    Bad state")
    else: 
        print("    Good state")

ui.observe(my_observer, names="_value")

In [12]:
ui

AutoUi(children=(SaveButtonBar(children=(ToggleButton(value=False, button_style='success', disabled=True, icon…

In [13]:
ui.value = dict(radius=2, inner_annulus=7, outer_annulus=8)

My observer call 1
{'radius': 2, 'inner_annulus': 7, 'outer_annulus': 8}
    Good state
